<a href="https://colab.research.google.com/github/OlegLaiok/Machine_Learning/blob/main/Lab7_2_Layok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # FashionMNIST

 ## Задание 7.2

<img src="https://emiliendupont.github.io/imgs/mnist-chicken/mnist-and-fashion-examples.png">

На практике построим нейронную сеть для классификации картиной. В качестве входных данных используются картинки одежды (в градациях серого, то есть цветовых каналов не 3, а 1). По входной картинке нужно предсказать тип одежды.

In [ ]:
!pip install torchvision

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import torch
import torchvision
from torchvision import transforms
from torch import nn, optim
import torch.nn.functional as F

Скачаем обучающую и контрольную выборки.

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = tuple(str(i) for i in range(10))

Каждая картинка имеет класс от 0 до 9, расшифровка меток класса:  

|class_id|class_name|
|----|----|
|0| T-shirt/top|
|1| Trouser|
|2| Pullover|
|3| Dress|
|4| Coat|
|5| Sandal|
|6| Shirt|
|7| Sneaker|
|8| Bag|
|9| Ankle boot| 

<h2 style="text-align: center;"><b>Задание</b></h2>

Посмотрите на датасет.
* Сколько картинок в train? В test?
* Равномерно ли распределено количество объектов разных классов в train датасете (это можно понять, поработав с полем `train_data` у переменной `trainset`)? в test?
* Выведите несколько картинок с помощью `pyplot`

Используя материалы семинаров по многослойным  и свёрточным сетям, напишите:
1.  Многослойную нейронную сеть MLP (Multi-Layer Perceptron) классифицирующую этот датасет
2. Свёрточную сеть, классифицирующую этот датасет
3. Сравните их качество (по графикам лосса и по Accuracy Score)

Если качество, которое получается у  нейросети, выше 95%, то Вы решили задачу с точностью, которую можно использовать в жизни.

In [ ]:
trainset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [ ]:
testset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test

In [ ]:
from tqdm import notebook

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 10)
        

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
MLP_network = MLP()

loss_f = nn.CrossEntropyLoss() 
optimizer = optim.Adam(MLP_network.parameters(), lr=0.0001)

for epoch in notebook.tqdm(range(5)):

    running_loss = 0.0
    for i, batch in enumerate(notebook.tqdm(trainloader)):
        # так получаем текущий батч
        X_batch, y_batch = batch
        
        # обнуляем веса
        optimizer.zero_grad()

        # forward + backward + optimize
        y_pred = MLP_network(X_batch)
        loss = loss_f(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        # выведем текущий loss
        running_loss += loss.item()
        # выведем качество каждые 2000 батчей
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Обучение закончено')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/15000 [00:00<?, ?it/s]

[1,  2000] loss: 0.879
[1,  4000] loss: 0.547
[1,  6000] loss: 0.514
[1,  8000] loss: 0.476
[1, 10000] loss: 0.471
[1, 12000] loss: 0.438
[1, 14000] loss: 0.420


  0%|          | 0/15000 [00:00<?, ?it/s]

[2,  2000] loss: 0.402
[2,  4000] loss: 0.394
[2,  6000] loss: 0.392
[2,  8000] loss: 0.368
[2, 10000] loss: 0.364
[2, 12000] loss: 0.362
[2, 14000] loss: 0.365


...

[3,  2000] loss: 0.340
[3,  4000] loss: 0.339
[3,  6000] loss: 0.325
[3,  8000] loss: 0.329
[3, 10000] loss: 0.339
[3, 12000] loss: 0.330
[3, 14000] loss: 0.329


  0%|          | 0/15000 [00:00<?, ?it/s]

[4,  2000] loss: 0.313
[4,  4000] loss: 0.324
[4,  6000] loss: 0.301
[4,  8000] loss: 0.295
[4, 10000] loss: 0.307
[4, 12000] loss: 0.312
[4, 14000] loss: 0.304


  0%|          | 0/15000 [00:00<?, ?it/s]

[5,  2000] loss: 0.294
[5,  4000] loss: 0.293
[5,  6000] loss: 0.286
[5,  8000] loss: 0.280
[5, 10000] loss: 0.292
[5, 12000] loss: 0.282
[5, 14000] loss: 0.287
Обучение закончено


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        y_pred = MLP_network(images)
        _, predicted = torch.max(y_pred, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
total_acc=0
for i in range(10):
  total_acc+=class_correct[i] / class_total[i]
total_acc = (total_acc / 10) * 100
print('Total accuracy is ', total_acc)

Accuracy of     0 : 89 %
Accuracy of     1 : 96 %
Accuracy of     2 : 73 %
Accuracy of     3 : 88 %
Accuracy of     4 : 83 %
Accuracy of     5 : 96 %
Accuracy of     6 : 63 %
Accuracy of     7 : 95 %
Accuracy of     8 : 97 %
Accuracy of     9 : 95 %
Total accuracy is  88.02


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        # вызов конструктора родителя
        super(ConvNet, self).__init__()
        # необходмо заранее знать, сколько каналов у картинки (сейчас = 1),
        # которую будем подавать в сеть, больше ничего
        # про входящие картинки знать не нужно
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, padding=1)
        # out_channels - это количество фильтров, которые применятся для свёртки
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=5, padding=1)
        #self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, padding=1)
        self.fc1 = nn.Linear(5 * 5 * 40, 500)
        self.fc2 = nn.Linear(500, 250)
        self.fc3 = nn.Linear(250, 125)
        self.fc4 = nn.Linear(125, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        x = x.view(-1, 5 * 5 * 40)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
# объявляем сеть
Conv_net = ConvNet()

# выбираем функцию потерь
loss_fn = torch.nn.CrossEntropyLoss()

# выбираем алгоритм оптимизации и learning_rate
learning_rate = 1e-4
optimizer = torch.optim.Adam(Conv_net.parameters(), lr=learning_rate, betas=(0.9,0.99), eps=1e-08, weight_decay=0.001)

# итерируемся
for epoch in notebook.tqdm(range(10)):

    running_loss = 0.0
    for i, batch in enumerate(notebook.tqdm(trainloader)):
        # так получаем текущий батч
        X_batch, y_batch = batch
        
        # обнуляем веса
        optimizer.zero_grad()

        # forward + backward + optimize
        y_pred = Conv_net(X_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        # выведем текущий loss
        running_loss += loss.item()
        # выведем качество каждые 2000 батчей
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Обучение закончено')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15000 [00:00<?, ?it/s]

[1,  2000] loss: 1.000
[1,  4000] loss: 0.676
[1,  6000] loss: 0.596
[1,  8000] loss: 0.542
[1, 10000] loss: 0.502
[1, 12000] loss: 0.485
[1, 14000] loss: 0.482


  0%|          | 0/15000 [00:00<?, ?it/s]

[2,  2000] loss: 0.419
[2,  4000] loss: 0.399
[2,  6000] loss: 0.394
[2,  8000] loss: 0.365
[2, 10000] loss: 0.378
[2, 12000] loss: 0.362
[2, 14000] loss: 0.361


...

[8,  2000] loss: 0.245
[8,  4000] loss: 0.227
[8,  6000] loss: 0.236
[8,  8000] loss: 0.225
[8, 10000] loss: 0.231
[8, 12000] loss: 0.227
[8, 14000] loss: 0.232


  0%|          | 0/15000 [00:00<?, ?it/s]

[9,  2000] loss: 0.210
[9,  4000] loss: 0.219
[9,  6000] loss: 0.219
[9,  8000] loss: 0.229
[9, 10000] loss: 0.228
[9, 12000] loss: 0.236
[9, 14000] loss: 0.220


  0%|          | 0/15000 [00:00<?, ?it/s]

[10,  2000] loss: 0.198
[10,  4000] loss: 0.214
[10,  6000] loss: 0.217
[10,  8000] loss: 0.222
[10, 10000] loss: 0.209
[10, 12000] loss: 0.224
[10, 14000] loss: 0.216
Обучение закончено


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        y_pred = Conv_net(images)
        _, predicted = torch.max(y_pred, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
total_acc=0
for i in range(10):
  total_acc+=class_correct[i] / class_total[i]
total_acc = (total_acc / 10) * 100
print('Total accuracy is ', total_acc)

Accuracy of     0 : 91 %
Accuracy of     1 : 97 %
Accuracy of     2 : 90 %
Accuracy of     3 : 93 %
Accuracy of     4 : 83 %
Accuracy of     5 : 93 %
Accuracy of     6 : 61 %
Accuracy of     7 : 98 %
Accuracy of     8 : 96 %
Accuracy of     9 : 95 %
Total accuracy is  90.17000000000002
